# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [27]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

nav = webdriver.Chrome()

#pesquisar por cotação do dólar
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#pegar a cotação do dólar
cotacao_dolar = nav.find_element_by_xpath('/html/body/div[7]/div/div[9]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div[1]/div/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

#pesquisar por cotação do euro
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#pegar a cotação do dólar
cotacao_euro = nav.find_element_by_xpath('/html/body/div[7]/div/div[9]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div[1]/div/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#pesquisar por cotação do ouro -> https://www.melhorcambio.com/
nav.get("https://www.melhorcambio.com/")
aba_original = nav.window_handles[0]
#achar o icone
nav.find_element_by_xpath('/html/body/div[15]/div[2]/div/table[2]/tbody/tr[2]/td[2]/a/img').click()
#mudar a aba
aba_nova = nav.window_handles[1]
nav.switch_to.window(aba_nova)
#pega a cotacao
#pegar pelo id
cotacao_ouro = nav.find_element_by_id('comercial').get_attribute('value')
#pegar pelo xpath
#cotacao_ouro = cotacao_euro = nav.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/input[2]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)


#encerra
nav.quit()

5.337899999999999
6.473004435
304.29


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [28]:
import pandas as pd

df = pd.read_excel('Produtos.xlsx')
display(df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [33]:
#alterando os preços
df.loc[df['Moeda'] == 'Dólar', ['Cotação']] = float(cotacao_dolar)
df.loc[df['Moeda'] == 'Euro', ['Cotação']] = float(cotacao_euro)
df.loc[df['Moeda'] == 'Ouro', ['Cotação']] = float(cotacao_ouro)

#calculando o preco total
df['Preço Base Reais'] = df['Cotação'] * df['Preço Base Original']
df['Preço Total'] = df['Ajuste'] * df['Preço Base Reais']

#visualizando as alterações
display(df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final,Preço Total
0,Câmera Canon,999.99,Dólar,5.337900,5337.846621,1.40,6999.930,7472.985269
1,Carro Renault,4500.00,Euro,6.473004,29128.519958,2.00,54000.000,58257.039915
2,Notebook Dell,899.99,Dólar,5.337900,4804.056621,1.70,7649.915,8166.896256
3,IPhone,799.00,Dólar,5.337900,4264.982100,1.70,6791.500,7250.469570
4,Carro Fiat,3000.00,Euro,6.473004,19419.013305,1.90,34200.000,36896.125279
5,Celular Xiaomi,480.48,Dólar,5.337900,2564.754192,2.00,4804.800,5129.508384
6,Joia 20g,20.00,Ouro,304.290000,6085.800000,1.15,8050.000,6998.670000


### Agora vamos exportar a nova base de preços atualizada

In [34]:
#cria um outro arquivo excel com os valores atualizado
df.to_excel("Produtos Atualizado.xlsx" , index = False)